In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import random

In [2]:
original_playlist_id = 'PLlUyZfiSZ0nU5EczlYUQ4i9dSO0Io1qD3' 
title = "Shuffled Roadtrip"
description = "This is a shuffled version of the roadtrip playlist"

In [3]:
def authenticate():
    flow = InstalledAppFlow.from_client_secrets_file(
        'client_secret.json',
        ['https://www.googleapis.com/auth/youtube.force-ssl']
    )
    credentials = flow.run_local_server(port=0)
    youtube = build('youtube', 'v3', credentials=credentials)
    return youtube

In [5]:
def get_playlist_items(youtube, playlist_id):
    all_videos = []
    request = youtube.playlistItems().list(
        part="snippet",
        maxResults=50,
        playlistId=playlist_id
    )
    
    while request is not None:
        response = request.execute()
        all_videos.extend(response['items'])
        request = youtube.playlistItems().list_next(request, response)
        
    return all_videos

In [6]:
def create_playlist(youtube, title, description):
    request = youtube.playlists().insert(
        part="snippet,status",
        body={
            "snippet": {
                "title": title,
                "description": description
            },
            "status": {
                "privacyStatus": "private"
            }
        }
    )
    response = request.execute()
    return response['id']

def add_items_to_playlist(youtube, playlist_id, videos):
    for video in videos:
        try:
            request = youtube.playlistItems().insert(
                part="snippet",
                body={
                    "snippet": {
                        "playlistId": playlist_id,
                        "resourceId": {
                            "kind": "youtube#video",
                            "videoId": video['snippet']['resourceId']['videoId']
                        }
                    }
                }
            )
            response = request.execute()
        except:
            print("Error adding video: " + video['snippet']['title'])

In [7]:
def shuffle_playlist(youtube, original_playlist_id, title, description):
    videos = get_playlist_items(youtube, original_playlist_id)
    random.shuffle(videos)
    new_playlist_id = create_playlist(youtube, title, description)
    add_items_to_playlist(youtube, new_playlist_id, videos)


In [8]:
youtube = authenticate()
shuffle_playlist(youtube, original_playlist_id, title, description)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=162388519786-0pdvs0po6iph552m9h8m5b5vl1g8kr3l.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60761%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=OqQL50eD04pqH8ZsPRRofKNUPlwldg&access_type=offline


HttpError: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/playlistItems?part=snippet&alt=json returned "Precondition check failed.". Details: "[{'message': 'Precondition check failed.', 'domain': 'global', 'reason': 'failedPrecondition'}]">